# biLSTM-LAN模型来实现

In [15]:
from Model import BiLstm_Lan_Trainer
from Data import CATEGORY
from keras.callbacks import EarlyStopping

BATCH_SIZE = 16
EPOCH = 50

model = BiLstm_Lan_Trainer(category_count = len(CATEGORY)+1,
                         seq_len = train_X.shape[1],
                         lstm_units=[256,256],
                         vocab_size = emb_matrix.shape[0],
                         emb_matrix = emb_matrix).build()

early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')

print('开始训练啦！！')
print(20*"===")
history = model.fit(train_X,train_Y,batch_size=BATCH_SIZE,
                    epochs = EPOCH,
                    class_weight="auto",
                    callbacks = [early_stopping],
                    validation_data = (val_X,val_Y)
                    )

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 90)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 90, 256)      590080      input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 90, 256)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 90, 512)      1050624     dropout_1[0][0]                  
____________________________________________________________________________________________

KeyboardInterrupt: 

time: 17.3 s


## 预测结果

In [9]:
preds = model.predict(test_X, batch_size=16, verbose=True)
from Evaluator import merge_preds,f1_score
from Prepare_sents import Sentences
testset = pickle.load(open('pickle_data/testset.pkl','rb'))
pre_docs = merge_preds(testset,preds,70,10)
source_docs = testset.docs
f1,prediction,recall = f1_score(pre_docs,source_docs,'all')
print("【严格相交】F1:{:.4f}  -  Predicition:{:.4f}  -  Recall:{:.4f}".format(f1,prediction,recall))

f1,prediction,recall = f1_score(pre_docs,source_docs,'others')
print("【不严格相交】F1:{:.4f}  -  Predicition:{:.4f}  -  Recall:{:.4f}".format(f1,prediction,recall))

7161/7161 [==============================] - 59s 8ms/step
【严格相交】F1:0.7369  -  Predicition:0.6846  -  Recall:0.7979
【不严格相交】F1:0.8043  -  Predicition:0.7472  -  Recall:0.8710
time: 1min 5s
